In [1]:
import xmltodict
import numpy as np
import json
import os
from pprint import pprint

In [11]:
def convert_valtype(valtype):
    # conversion from Cyclus valtype [key]
    # to Workbench ValType [val]
    conversion_dict = {'string': 'String',
                       'nonNegativeInteger': 'Int',
                       'boolean': 'Int',
                       'double': 'Real',
                       'positiveInteger': 'Int',
                       'float': 'Real',
                       'double': 'Real',
                       'duration': 'Int',
                       'integer': 'Int',
                       'nonPositiveInteger': 'Int',
                       'negativeInteger': 'Int',
                       'long': 'Real',
                       'int': 'Int',
                       }

    additional_dict = {'nonNegativeInteger': {'MinValInc': 0},
                       'positiveInteger': {'MinValExc': 0},
                       'boolean': {'ValEnums': [0, 1]},
                       'duration': {'MinValInc': 0},
                       'nonPositiveInteger': {'MaxValInc': 0},
                       'negativeInteger': {'MaxValExc': 0},
                       }
    d = {'valType': conversion_dict[valtype]}
    if valtype in additional_dict:
        for key, val in additional_dict[valtype].items():
            d[key] = val
    return d

def convert_num_limits(limit_type):
    conversion_dict = {'oneOrMore': {'ChildAtLeastOne': True},
                       'zeroOrMore': {'MinOccurs': 0}
                       }


def change_element(element_dict):
    d = {element_dict['@name']: convert_valtype(element_dict['data']['@type'])}
    

In [15]:
i = x['grammar']['start']['element']['interleave']['element']
ctrl = i[0]
recipe = x['grammar']['start']['element']['interleave']['zeroOrMore'][1]
"""
def fui_generator(tree):
    fui_str = ''
    fui_str += '"%s"{\n' %tree['@name']
    fui_str += '\ttype=object\n'
    # get essential variables first
    for entry in tree['interleave']['element']:
        options = parse_options(entry)
        fui_str += '\t%s{\n\t\t%s\n\t\t}\n' %(entry['@name'], '\n\t\t'.join(options))
    # optional variables
    for entry in tree['interleave']['optional']:
        entry = entry['element']
        options = parse_options(entry)
        options.append('need=optional')
        fui_str += '\t%s{\n\t\t%s\n\t\t}\n' %(entry['@name'], '\n\t\t'.join(options))
    print(fui_str)
""" 
def convert_valtype(val):
    if 'string' in val.lower() or 'str' in val.lower():
        return 'string'
    else:
        return 'real'

def parse_options(entry_dict):
    option_list = []

    if 'data' not in entry_dict:
        if 'text' in entry_dict.keys():
            return ['type=string']
        else:
            return []
    for key, val in entry_dict['data'].items():
        if key == '@type':
            option_list.append('type=%s' %convert_valtype(val))
        else:
            print('what else %s' %key)
    
    return option_list
        

In [95]:
###
# generate highlight files
##
##
rgb_dict = {'black': [0,0,0],
            'white': [255, 255, 255],
            'red': [255, 0, 0],
            'lime': [0, 255, 0],
            'blue': [0, 0, 255],
            'yellow': [255, 255, 0],
            'cyan': [0, 255, 255],
            'magenta': [255, 0, 255],
            'silver': [192, 192, 192]}
def highlight_maker(name, word, color='blue'):
    s = """rule("%s") {
    pattern = "%s"
    bold = true
    foreground {
        red = %i
        green = %i
        blue = %i
    }
}
    
    """ %(name, word, rgb_dict[color][0], rgb_dict[color][1], rgb_dict[color][2])
    return s
highlight_str = ''
for i in ['simulation', 'control', 'archetypes',
          'facility', 'region', 'recipe']:
    highlight_str += highlight_maker(i, i)
# highlight_str += highlight_maker('brack_open', '{', 'red')
# highlight_str += highlight_maker('brack_close', '}', 'red')
# highlight_str += highlight_maker('square_open', '[', 'lime')
# highlight_str += highlight_maker('square_close', ']', 'lime')
highlight_str += '''rule("Quoted string") {
pattern = """'[^']*'|"[^"]*""""
bold = true
foreground {
red = 255
green = 130
blue = 0
}
background {
red = 255
green = 130
blue = 0
alpha = 25
}
}

rule("equal"){
pattern="="
background{
red=192
green=192
blue=192
}
}

rule("comment"){
pattern="%*"
foreground{
red=0
green=255
blue=0
}
}
'''

In [96]:
###
# generate schema files
##
## This is manually written because there are so many unnecessary things that are not needed
## things like the commodity block are goners
sch_str = """simulation{
    Description="Agent-based fuel cycle simulator"
    InputTmpl="init_template"
    control {
             MinOccurs=1
             Description="Defines simulation time and decay methods"
             MaxOccurs=1
             duration={
                 MinOccurs=1
                 MaxOccurs=1
                 Description="the number of timesteps in simulation"
                 ValType=Int
                 }
             startyear={
                 MinOccurs=1
                 MaxOccurs=1
                 Description="the year to start the simulation"
                 ValType=Int
                 }
             startmonth={
                 MinOccurs=1
                 MaxOccurs=1
                 Description="the month to start the simulation"
                 ValType=Int
                 ValEnums=[ 1 2 3 4 5 6 7 8 9 10 11 12 ]
                 }
             decay={
                 MinOccurs=0
                 MaxOccurs=1
                 Description="How to model decay in Cyclus"
                 ValType=String
                 ValEnums=["lazy" "manual" "never"]
                 }
             dt={
                 MinOccurs=0
                 MaxOccurs=1
                 ValType=Real
                 Description="duration of a single timestep in seconds"
                 }
             explicit_inventory={
                 MinOccurs=0
                 MaxOccurs=1
                 ValType=Int
                 ValEnums=[0 1]
                 Description="boolean specifying whether or nor to track inventory in each agent"
                 }
             
            }

    archetypes {
        MinOccurs=1
        Description="Defines the archetypes used in this simulation"
        MaxOccurs=1
        spec={
            MinOccurs=1
            lib={MinOccurs=1
                 MaxOccurs=1
                 ValType=String
                }
            name={MinOccurs=1
                 MaxOccurs=1
                 ValType=String
                 }
            }
    }
    
    facility {
        Description="Facility definition block"
        MinOccurs=1
        %facility_schema
    }
    
    region{
        Description="Region definition block"
        MinOccurs=1
        %region_schema
    }
    
    recipe{
        Description="Recipe definition block"
        name={
            MinOccurs=1
            MaxOccurs=1
            ValType=String
            }
        basis={
            MinOccurs=1
            MaxOccurs=1
            ValType=String
            ValEnums=["mass" "atom"]
            }
        nuclide={
            MinOccurs=1
            id={MinOccurs=1 MaxOccurs=1}
            comp={MinOccurs=1 MaxOccurs=1 ValType=Real}
            }
    }

}
"""


In [97]:
###
# generate initial template file
##
##
init_template = """simulation{

    control {
        duration = 1234
        startmonth = 1
        startyear = 2020
        explicit_inventory=0
        dt=2629846
        decay="lazy"
    }
    
    archetypes {
        spec ["this will be automatically filled"]
    }

    facility {
        "write your facilities here"
    }
    
    region {
        "write your regions here"
    }
    
    recipe {
        "write your recipes here"
    }
}"""

template_dict = {'init_template': init_template}
# archetypes spec automatically filled from distribution

In [98]:
###
# generate grammar files
##
##

schema_path = '/Users/4ib/Desktop/git/cyclus_gui/neams/cyclus.sch'
template_dir = '/Users/4ib/Desktop/git/cyclus_gui/neams/templates/'
highlight_path = '/Users/4ib/Desktop/git/cyclus_gui/neams/cyclus.wbh'

grammar_str = """name= Cyclus
enabled = true

parser = waspson
schema = "%s"
validator = wasp

templates = "%s"

highlighter = "%s"

extensions = [cyclus]
""" %(schema_path, template_dir, highlight_path)


grammar_path = '/Users/4ib/.workbench/2.0.0/grammars/cyclus.wbg'

with open(grammar_path, 'w') as f:
    f.write(grammar_str)

with open(schema_path, 'w') as f:
    f.write(sch_str)
    
with open(highlight_path, 'w') as f:
    f.write(highlight_str)

for key, val in template_dict.items():
    with open(os.path.join(template_dir, key+'.tmpl'), 'w') as f:
        f.write(val)

In [100]:
! open ~/Downloads/Workbench-Darwin/bin/Workbench

1. put the grammar file into .workbench/2.0.0/grammars/*.wbh
